# recap с предыдущего занятия

- у каждого* объекта  атрибут `__dict__` -- словарь с атрибутами
- базовое поведение при записи `some_thing.attr = another_thing`
  - в итоге приводит к
  - `some_thing :: __dict__['attr'] is another_thing`
- как читаются атрибуты?
  - `value = some_thing.attr`
  - если some_thing -- объект (не класс)
      - ищем в `some_thing :: __dict__['attr']`
      - ищем в `some_thing.__class__ :: __dict__['attr']`
      - ищем по всем классам, в порядке наследования (MRO -- method resolution order)
  - если some_thing -- класс
      - ищем в `some_thing.__class__ :: __dict__['attr']`
      - ищем по всем классам, в порядке наследования (MRO -- method resolution order)

In [3]:
class SuperClass:
    a = 10
    lst = []
    
    def __init__(self):
        self.a += 1
        self.lst += ['added_from__so_something_with_a']
        
#         self.attr += val
        # self.attr = self.attr.__iadd__(val)
    
#         self.a += 1
#         self.a = self.a.__iadd__(1)

#         self.lst += whatever
#         self.lst = self.lst.__iadd__(whatever)

instance = SuperClass()

assert SuperClass.__dict__['a'] == 10
assert instance.__dict__['a'] == 11

assert instance.lst is SuperClass.__dict__['lst'] is instance.__dict__['lst']
assert SuperClass.lst == ['added_from__so_something_with_a']


# Удаление атрибутов 

`del some_thing.attr`
Удаление работает как **запись** 

In [5]:
class Base:
    base_attr = 10

class Child(Base):
    pass

child = Child()

In [7]:
del child.base_attr

AttributeError: base_attr

In [8]:
del Child.base_attr

AttributeError: base_attr

In [9]:
del Base.base_attr

# На самом деле, имя атрибута может быть динамическим

## чтение `getattr`

In [11]:
class Base:
    base_attr = 10

class Child(Base):
    pass

child = Child()

print(
    child.base_attr
)
print(
    getattr(child, 'base_attr')
)

10
10


In [12]:
getattr(child, 'non_existing_attribute')

AttributeError: 'Child' object has no attribute 'non_existing_attribute'

In [15]:
obj = object()

In [16]:
getattr(child, 'non_existing_attribute', obj) is obj  # Вспоминаем про sentinel object pattern!


True

## проверяем, что атрибут есть `hasattr`

In [19]:
hasattr(child, 'base_attr')

True

In [20]:
hasattr(child, 'non_existing_attribute')

False

In [ ]:
if hasattr(instance, 'attr_name'):
    instance.attr_name

**под капотом все равно вызывает `getattr`!** (Ествественно, потому что логика поиска сложная)

In [ ]:
class MyTable:
    id = IntegerField()
    username = CharField()
    
instance = MyTable
instance.id = 10

In [ ]:
getattr(instance, 'id')

## запись `setattr`

In [ ]:
# obj.name = value
setattr(obj, name, value)

## удаление delattr

In [ ]:
# del obj.name
delattr(obj, 'name')

# Как посмотреть **все**  атрибуты

In [17]:
SuperClass().__dict__

{'a': 11,
 'lst': ['added_from__so_something_with_a', 'added_from__so_something_with_a']}

In [22]:
dir(SuperClass())

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'lst']

# Приватные атрибуты

https://google.github.io/styleguide/pyguide.html

In [19]:
class ListWrapper:
    def __init__(self, list_instance):
        self._list = list_instance
        self.__super_private_attribute = list_instance

lw = ListWrapper([])
lw._list
lw.__super_private_attribute

AttributeError: 'ListWrapper' object has no attribute '__super_private_attribute'

In [21]:
lw._ListWrapper__super_private_attribute

[]

## Что когда использовать?

- по умолчанию все делаем приватным `_attr`
- если атрибут все-таки должен быть доступен **КАК ПУБЛИЧНОЕ АПИ** -- делаете пуличным без подчеркивания
- `__` -- НЕ ДЕЛАЕМ -- будете знать, когда надо

# Магические атрибуты
- https://docs.python.org/3/reference/datamodel.html
- https://docs.python.org/3/glossary.html

# __del__

In [23]:
class Foo:
    def __del__(self):
        print('__del__', id(self))

In [24]:
Foo()

In [28]:
lst = [Foo()]

In [29]:
for value in lst:
    print(value)

In [31]:
value

In [30]:
del lst[0]

In [32]:
del value

# __repr__, __str__

In [63]:
class Foo:
    def __init__(self, a):
        self._a = a
    def __str__(self):
        return '__str__ called'
    def __repr__(self):
        return f'Foo(a={self._a!r})'
    def __eq__(self, rhs):
        if not isinstance(rhs, Foo):
            return NotImplemented
        return self._a == rhs._a
    

In [66]:
from dataclasses import dataclass

In [71]:
@dataclass
class Foo:
    a: int

In [75]:
foo = Foo('10')

In [76]:
str(foo)

"Foo(a='10')"

In [77]:
repr(foo)

"Foo(a='10')"

In [78]:
Foo(1) == Foo(1)

True

In [ ]:
# a == b
# a.__eq__(b) # NotImplemented
# b.__eq__(a) # NotImplemented
# object.__eq__(a, b)
# object.__eq__(b, a)
# a is b

In [64]:
foo = Foo(a=10)

In [65]:
foo == eval(repr(foo))

True

In [56]:
'{a!r}'.format(a='1')

"'1'"

In [57]:
'{a!s}'.format(a='1')

'1'

In [ ]:
'{a}'

In [40]:
str(foo)

'__str__ called'

In [41]:
repr(foo)

'__repr__ called'

In [43]:
foo = Foo(*args, **kwargs)

NameError: name 'args' is not defined

In [ ]:
foo == eval(repr(foo))

In [ ]:
foo.__str__

In [44]:
print(1)

1


In [45]:
str(1)

'1'

In [46]:
str('1')

'1'

In [48]:
type(
    eval(str('1'))
)

int

In [49]:
repr(1)

'1'

In [50]:
repr('1')

"'1'"

In [51]:
eval(repr('1'))

'1'

In [52]:
eval(repr(1))

1

In [36]:
object.__str__(foo)

'<__main__.Foo object at 0x7f915964d730>'

In [37]:
object.__repr__(foo)

'<__main__.Foo object at 0x7f915964d730>'

# MRO

https://en.wikipedia.org/wiki/C3_linearization

Основной посыл:
- Родитель всгда идет раньше детей
- Левый родитель всегда идет раньше правого родителя

Пусть  `Type` -- класс, `parent_i` -- упорядоченное множество типов родителей (прямые родители) класса `Type`

Тогда линеаризация 
```
L(Type) = Type + MERGE(
    L(parent_1),  # линеаризация первого родителя
    L(parent_2),  # линеаризация второго родителя
    ...,
    L(parent_N),  # линеаризация последнего родителя
    [parent_1, parent_2, ..., parent_N]  # упорядоченный список самих родителей
)
```

где `MERGE` итеративная процедура, где на каждой итерации происходит следующее:

- последовательно (от первого к последнему) идет по спискам
- берет тип -- голову списка (первый элемент)
- проверяет, встречается ли этот тип где-то, кроме как на первом месте в других списках
- если тип встречается только в голове (или не встречается), то тип добавляется в итоговую линеаризацию и вычеркивается из всех списков внутри MERGE, переходим к следующей итерации
- если тип встречается НЕ В ГОЛОВЕ, то переходим к следующему списку, берем голову, ...

## Diamond пример

In [ ]:
     Base
Left      Right
      Bottom

In [83]:
class Base: pass
class Left(Base):
    pass
#     def overloaded_method(self):
#         super(Left, self).overloaded_method()
class Right(Base):
    pass
#     def overloaded_method(self):
#         super(Right, self).overloaded_method()
class Bottom(Left, Right): pass

In [84]:
Base.mro()

[__main__.Base, object]

In [80]:
Bottom.mro()

[__main__.Bottom, __main__.Left, __main__.Right, __main__.Base, object]

In [ ]:
L(type) = type + MERGE(*(L(p_i) for i), parents)

In [ ]:
L(object) = object

In [ ]:
L(Base) = Base + MERGE(
    L(object),  # parent linearization
    [object,], # parents
) = Base + MERGE(
    [object],
    [object],
) = Base + object = [Base, object]

In [ ]:
L(Left) = Left + MERGE(
    L(Base) = [Base, object],
    [Base],
) = [Left, Base, object]

L(Right) = [Right, Base, object]

In [ ]:
L(Bottom) = Bottom + MERGE(
    L(Left),
    L(Right),
    [Left, Right],
) = Bottom + MERGE(
    [Left, Base, object],
    [Right, Base, object],
    [Left, Right],
) = Bottom + Left + MERGE(
    [Base, object],
    [Right, Base, object],
    [Right]
) = Bottom + Left + Right + MERGE(
    [Base, object],
    [Base, object],
) = [Bottom, Left, Right, Base, object]

In [85]:
Bottom.mro()

[__main__.Bottom, __main__.Left, __main__.Right, __main__.Base, object]

## Более сложный пример

```
A         E             F
x        x  xx        x
 x       x   xxx    xx
 xx     xx     x  xxx
  xx   x       xxxx
  xx B          D               G
     x          x           xxx
     xxx       xx       xxxx
       xx     xx     xxx
        xx   xx  xxxxx
           C x xxx
```

In [104]:
class A: pass
class E: pass
class F: pass
class B(A, E): pass
class D(E, F): pass
class G: pass
class C(B, D, G): pass

In [105]:
C.mro()

[__main__.C,
 __main__.B,
 __main__.A,
 __main__.D,
 __main__.E,
 __main__.F,
 __main__.G,
 object]

Примерная логика:
- идем всегда наверх влево, затем вправо, до тех пор, пока какой-то родитель не станет зависеть от этого правого
- C -> B -> A
- дальеш должен был быть E, но E родитель D, который по определению должен идти перед
- A -> D
- идем вверх влево D -> E
- вправо E -> F (можем, мотому что от F никто, кроме D не наследуется, а D уже был
- F -> G

In [106]:
for clazz in C.mro():
    print(clazz.__name__, clazz.mro())

C [<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class '__main__.G'>, <class 'object'>]
B [<class '__main__.B'>, <class '__main__.A'>, <class '__main__.E'>, <class 'object'>]
A [<class '__main__.A'>, <class 'object'>]
D [<class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class 'object'>]
E [<class '__main__.E'>, <class 'object'>]
F [<class '__main__.F'>, <class 'object'>]
G [<class '__main__.G'>, <class 'object'>]
object [<class 'object'>]


## Пример, когда MRO не вычислим

In [107]:
class Base: pass
class Left(Base): pass
class Right(Base): pass
class Middle(Left, Right): pass
class Bottom(Middle, Right, Left): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Left, Right

- Bottom явно наследуется от Right, Left. Это значит, что Right должен идти ДО Left
- В то же время, он наследуется от Middle, который наследуется от Left, Right.
  - это значит, что Right должен идти ПОСЛЕ Left
- противоречие

# Миксины

In [ ]:
class Foo(FullOrderMixin):
    def __eq__
    def __lt__

In [ ]:
class FullOrderMixin:
    def __ne__
    def __le__

In [ ]:
class Foo(Foo2, Foo3):
    pass

In [ ]:
self.logger()

In [90]:
import logging

In [95]:
class LoggingMixin:
    def logger(self):
        if not hasattr(self, '_logger'):
            self._logger = logging.getLogger(self.__class__.__name__)
        return self._logger

In [99]:
class Another:
    def logger(self):
        print('hahaha')

In [102]:
class Foo(Another, LoggingMixin):
    pass

In [103]:
Foo().logger().error('asdads')

hahaha


AttributeError: 'NoneType' object has no attribute 'error'